# mdi-PLS 在域自适应中的应用
## Dr. Ramin Nikzad-Langerodi
### Bottleneck Analytics GmbH
info@bottleneck-analytics.com

___
首先，我们加载一些将要使用的模块，包括 di-PLS 类。

In [ ]:
# 加载模块
import numpy as np
import matplotlib.pyplot as plt
from diPLSlib.models import DIPLS as dipls
from diPLSlib.utils import misc as fct
from matplotlib.gridspec import GridSpec

# 配置中文绑图支持
fct.setup_chinese_plot()

### 数据集

让我们创建一些模拟的“源域”和“目标域”数据集，分别包含 N=50 个样本和 p=100 个变量。

### 源域 (分析物 + 1 种干扰物)

In [ ]:
n = 50  # 样本数量
p = 100 # 变量数量

为此，我们定义了 4 个高斯函数，其中第一个对应于我们将尝试建模的（分析物）信号，另外三个对应于干扰信号（干扰物）。

源域数据集将仅包含分析物信息以及三个干扰物中**一个**的贡献。

In [ ]:
# 生成信号
S1 = fct.gengaus(p, 50, 15, 8, 0)  # 分析物
S2 = fct.gengaus(p, 70, 10, 10, 0) # 干扰物
S = np.vstack([S1,S2])

# 分析物浓度
Cs = 10*np.random.rand(n,S.shape[0])

# 光谱
X = Cs@S

# 随机噪声
noise = 0.005*np.random.rand(n,p)

# 源域光谱加上噪声
Xs = X# + noise

在目标域中，我们将有来自分析物和另外两个额外干扰物中一个的贡献。

In [ ]:
# 目标域 1 (分析物 + 2 种干扰物)
S1 = fct.gengaus(p, 50, 15, 8, 0)  # 分析物
S2 = fct.gengaus(p, 70, 10, 10, 0) # 干扰物 1
S3 = fct.gengaus(p, 30, 10, 10, 0) # 干扰物 2
S = np.vstack([S1,S2,S3])

# 分析物浓度
Ct1 = 10*np.random.rand(n,S.shape[0])

# 光谱
X = Ct1@S

# 随机噪声
noise = 0.005*np.random.rand(n,p)

# 目标域光谱加上噪声
Xt1 = X# + noise

# 目标域 2 (分析物 + 3 种干扰物)
S1 = fct.gengaus(p, 50, 15, 8, 0)  # 分析物
S2 = fct.gengaus(p, 70, 10, 10, 0) # 干扰物 1
S3 = fct.gengaus(p, 30, 10, 10, 0) # 干扰物 2
S4 = fct.gengaus(p, 75, 75, 150, 0) # 干扰物 3
S = np.vstack([S1, S2, S3, S4])

# 分析物浓度
Ct2 = 10*np.random.rand(n,S.shape[0])

# 光谱
X = Ct2@S

# 随机噪声
noise = 0.005*np.random.rand(n,p)

# 目标域光谱加上噪声
Xt2 = X# + noise

让我们绘制分析物和干扰物的纯信号以及模拟数据集。

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(211)
plt.plot(S1)
plt.plot(S2)
plt.plot(S3)
plt.plot(S4)
plt.legend(['分析物','干扰物 1','干扰物 2', '干扰物 3'])
plt.title('纯信号')
plt.xlabel('X-变量')
plt.ylabel('信号')
plt.axvline(x=50,linestyle='-',color='k',alpha=0.5)
plt.axvline(x=70,linestyle=':',color='k',alpha=0.5)
plt.axvline(x=30,linestyle=':',color='k',alpha=0.5)


plt.subplot(234)
plt.plot(Xs.T, 'b', alpha=0.2)
plt.title('源域')
plt.xlabel('X-变量')
plt.ylabel('信号')

plt.subplot(235)
plt.plot(Xt1.T, 'r', alpha=0.2)
plt.title('目标域 1')
plt.xlabel('X-变量')
plt.ylabel('信号')

plt.subplot(236)
plt.plot(Xt2.T, 'g', alpha=0.2)
plt.title('目标域 2')
plt.xlabel('X-变量')
plt.ylabel('信号')
plt.tight_layout()
plt.show()

### 偏最小二乘回归 (PLS)
我们现在将尝试在给定模拟光谱的情况下对源域中的分析物浓度进行建模，并使用标准 PLS 将模型应用于目标域。

In [ ]:
### 准备工作
nr_comp = 2                         # 组件数量
l = 0                               # 正则化参数
target_domains = [Xt1, Xt2]         # 目标域
ys = np.expand_dims(Cs[:, 0],1)     # 源域浓度
yt1 = np.expand_dims(Ct1[:, 0],1)   # 目标域 1 浓度
yt2 = np.expand_dims(Ct2[:, 0],1)   # 目标域 2 浓度

### 源域模型
m_pls = dipls(A=nr_comp, l=l)
m_pls.fit(Xs, ys, Xs, target_domains)
b_pls = m_pls.b_

yhat_plsT1 = m_pls.predict(Xt1)
yhat_plsT2 = m_pls.predict(Xt2)

error_plsT1 = fct.rmse(yt1, yhat_plsT1)
error_plsT2 = fct.rmse(yt2, yhat_plsT2)

min_ = -5
max_ = 50

# 绘制
plt.figure(figsize=(9, 4))
plt.subplot(121)
plt.scatter(yt1, yhat_plsT1, color='b', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('目标域 1 的预测')
plt.xlabel('测量值')
plt.ylabel('预测值')

plt.subplot(122)
plt.scatter(yt2, yhat_plsT2, color='b', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('目标域 2 的预测')
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.suptitle('PLS')
plt.tight_layout()


正如预期的那样，源域模型的预测在应用于目标域时不是很准确，因为后者包含来自额外干扰物的贡献。

### 多域不变偏最小二乘 (mdi-PLS)
我们现在将包含目标域数据（没有相应的分析物浓度），并添加一些域正则化，以便同时解释两个目标域中的额外干扰。

In [ ]:
# 准备工作
l = 100   # 正则化参数

### 针对目标域 1 的 mdi-PLS 模型
m_diplsT1 = dipls(A=nr_comp, l=l, target_domain=1)
m_diplsT1.fit(Xs, ys, Xs, target_domains)
b_diplsT1 = m_diplsT1.b_
yhat_diplsT1 = m_diplsT1.predict(Xt1)

error_diplsT1 = fct.rmse(yt1, yhat_diplsT1)

### 针对目标域 2 的 mdi-PLS 模型
m_diplsT2 = dipls(A=nr_comp, l=l, target_domain=2)
m_diplsT2.fit(Xs, ys, Xs, target_domains)
b_diplsT2 = m_diplsT2.b_
yhat_diplsT2 = m_diplsT2.predict(Xt2)

error_diplsT2 = fct.rmse(yt2, yhat_diplsT2)

min_ = -5
max_ = 15

# 绘制
plt.figure(figsize=(9, 4))
plt.subplot(121)
plt.scatter(yt1, yhat_diplsT1, color='m', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('目标域 1 的预测')
plt.xlabel('测量值')
plt.ylabel('预测值')

plt.subplot(122)
plt.scatter(yt2, yhat_diplsT2, color='m', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('目标域 2 的预测')
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.suptitle('mdi-PLS')
plt.tight_layout()

该模型现在可以在两个目标域中准确预测分析物浓度！请注意，我们仅包含了目标域的 X 数据而没有 Y 数据（连同源域的 X 和 Y 数据），就找到了适用于目标域的良好模型。值得注意的是，mdi-PLS 为每个目标域生成单独的模型。通过指定 'target_domain' 参数，我们可以确定要在哪个目标域中应用模型。
    
```python
m_diplsT1 = dipls(Xs, ys, Xs, target_domains, nr_comp)
m_diplsT1.fit(l=l, target_domain=1)
```

生成针对目标域 1 的模型，而

```python
m_diplsT2 = dipls(Xs, ys, Xs, target_domains, nr_comp)
m_diplsT2.fit(l=l, target_domain=2)
```
生成针对目标域 2 的模型。如果我们不知道测试数据是从哪个域采样的，建议使用 target_domain=0。

让我们试着理解 mdi-PLS 如何改进目标域的预测。为此，我们将查看源域和目标域数据在潜变量 (LV) 空间上的投影，该空间由基于所有域光谱的 PCA 模型和 mdi-PLS 模型涵盖。

In [ ]:
### PCA
X = np.vstack([Xs, Xt1, Xt2])
X = X[:,...] - np.mean(X, 0)
U,S,V = np.linalg.svd(X)
T = U[:, :100]@np.diag(S)

plt.figure(figsize=(9, 3))
plt.subplot(121)
a = plt.scatter(T[:50, 0], T[:50, 1], edgecolors='k')
el = fct.hellipse(T[:50, :2])
plt.plot(el[0,:],el[1,:])

b = plt.scatter(T[51:100, 0], T[51:100, 1], edgecolors='k')
el = fct.hellipse(T[51:100, :2])
plt.plot(el[0,:],el[1,:])

c = plt.scatter(T[101:, 0], T[101:, 1], edgecolors='k')
el = fct.hellipse(T[101:, :2])
plt.plot(el[0,:],el[1,:])

plt.xlabel('PC 1')
plt.ylabel('PC 2')

ax = plt.gca()
ax.axhline(y=0,color='k',linestyle=':')
ax.axvline(x=0,color='k',linestyle=':')

plt.legend([a, b, c], ['源域', '目标域 1', '目标域 2'])
plt.title('PCA')


### mdi-PLS
nr_comp = 2
l =  100
target_domains = [Xt1, Xt2]
ys = np.expand_dims(Cs[:, 0],1)
yt1 = np.expand_dims(Ct1[:, 0],1)
yt2 = np.expand_dims(Ct2[:, 0],1)

m_dipls = dipls(A=nr_comp, l=l, target_domain=0)
m_dipls.fit(Xs, ys, Xs, target_domains)

plt.subplot(122)
a = plt.scatter(m_dipls.Ts_[:, 0], m_dipls.Ts_[:, 1], edgecolors='k')
el = fct.hellipse(m_dipls.Ts_)
plt.plot(el[0,:],el[1,:])

b = plt.scatter(m_dipls.Tt_[0][:, 0], m_dipls.Tt_[0][:, 1], edgecolors='k')
el = fct.hellipse(m_dipls.Tt_[0])
plt.plot(el[0,:],el[1,:])

c = plt.scatter(m_dipls.Tt_[1][:, 0], m_dipls.Tt_[1][:, 1], edgecolors='k')
el = fct.hellipse(m_dipls.Tt_[1])
plt.plot(el[0,:],el[1,:])

ax = plt.gca()
ax.axhline(y=0,color='k',linestyle=':')
ax.axvline(x=0,color='k',linestyle=':')

plt.xlabel('LV 1')
plt.ylabel('LV 2')
plt.title('mdi-PLS')
plt.legend([a, b, c], ['源域', '目标域 1', '目标域 2'])

plt.tight_layout()
plt.show()


我们可以看到，源域和目标域得分（即在 LV 空间上的投影）在 mdi-PLS 模型中对齐得很好。在源域中使用得分投影看起来“就像是从相同的底层分布中采样的一样”的 LV 对分析物进行建模，可以提高跨域的泛化能力。这直接遵循了“不同域学习理论”（Ben-David 等人，Mach. Learn. 2010）。

现在让我们看看 PLS 和 mdi-PLS 模型的回归系数。

In [ ]:
fig = plt.figure(figsize=(9, 9), constrained_layout=True)
gs = GridSpec(4, 3, figure=fig)

# 以网格形式创建子图
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[0, 1], sharey=ax1)
ax3 = fig.add_subplot(gs[0, 2], sharey=ax1)
ax4 = fig.add_subplot(gs[1, 0])
ax5 = fig.add_subplot(gs[1, 1], sharey=ax4)
ax5_1 = fig.add_subplot(gs[1, 2], sharey=ax4)
ax6 = fig.add_subplot(gs[2, 0])
ax7 = fig.add_subplot(gs[2, 1:], sharey=ax6)
ax8 = fig.add_subplot(gs[3, 0])
ax9 = fig.add_subplot(gs[3, 1], sharey=ax8)
ax10 = fig.add_subplot(gs[3, 2], sharey=ax8)

# 原始数据
ax1.plot(Xs.T, 'b', alpha=0.2)
ax1.set_title('源域')
ax1.set_xlabel('X-变量')
ax1.set_ylabel('信号')

ax2.plot(Xt1.T, 'r', alpha=0.2)
ax2.set_title('目标域 1')
ax2.set_xlabel('X-变量')
ax2.set_ylabel('信号')

ax3.plot(Xt2.T, 'g', alpha=0.2)
ax3.set_title('目标域 2')
ax3.set_xlabel('X-变量')
ax3.set_ylabel('信号')

# 回归系数
ax4.plot(b_pls, 'b')
ax4.set_ylabel('回归系数')
ax4.set_xlabel('X-变量')
ax4.set_title('PLS 模型 (源域)')
ax4.axhline(y=0, color='k')

ax5.plot(b_diplsT1, 'r')
ax5.set_ylabel('回归系数')
ax5.set_xlabel('X-变量')
ax5.set_title('mdi-PLS 模型 (针对目标域 1)')
ax5.axhline(y=0, color='k')

ax5_1.plot(b_diplsT2, 'g')
ax5_1.set_ylabel('回归系数')
ax5_1.set_xlabel('X-变量')
ax5_1.set_title('mdi-PLS 模型 (针对目标域 2)')
ax5_1.axhline(y=0, color='k')

# 权重
ax6.plot(m_pls.W_[:, 0], 'b')
ax6.plot(m_pls.W_[:, 1], '--', color='b')
ax6.set_ylabel('权重')
ax6.legend(['LV1','LV2'])
ax6.set_xlabel('X-变量')
ax6.set_title('PLS 模型 (源域)')
ax6.axhline(y=0, color='k')

ax7.plot(m_diplsT1.W_[:, 0],'m')
ax7.plot(m_diplsT1.W_[:, 1], '--', color='m')
ax7.set_ylabel('权重')
ax7.legend(['LV1','LV2'])
ax7.set_xlabel('X-变量')
ax7.set_title('mdi-PLS 模型 (所有域)')
ax7.axhline(y=0, color='k')

# 载荷
ax8.plot(m_pls.Ps_[:, 0], 'b')
ax8.plot(m_pls.Ps_[:, 1], '--', color='b')
ax8.set_ylabel('载荷')
ax8.legend(['LV1','LV2'])
ax8.set_xlabel('X-变量')
ax8.set_title('PLS 模型 (源域)')
ax8.axhline(y=0, color='k')

ax9.plot(m_diplsT1.Pt_[0][:, 0],'r')
ax9.plot(m_diplsT1.Pt_[0][:, 1], '--', color='r')
ax9.set_ylabel('载荷')
ax9.legend(['LV1','LV2'])
ax9.set_xlabel('X-变量')
ax9.set_title('mdi-PLS 模型 (针对目标域 1)')
ax9.axhline(y=0, color='k')

ax10.plot(m_diplsT2.Pt_[1][:, 0], 'g')
ax10.plot(m_diplsT2.Pt_[1][:, 1], '--', color='g')
ax10.set_ylabel('载荷')
ax10.legend(['LV1','LV2'])
ax10.set_xlabel('X-变量')
ax10.set_title('mdi-PLS 模型 (针对目标域 2)')
ax10.axhline(y=0, color='k')

plt.tight_layout()
plt.show()


第一列显示了仅使用源域数据的普通 PLS 光谱、回归系数、权重和载荷，而第二列和第三列显示了包含两个目标域光谱的 mdi-PLS 模型的相同物理量。请注意，该模型具有一组共同的权重，但具有特定于每个目标域的不同载荷和回归系数。

让我们现在来看看 'target_domain' 参数如何影响目标域 2 中的预测。

In [ ]:
# 准备工作
l = 100   # 正则化参数
min_ = -5 # 用于绘图的最小值
max_ = 15 # 用于绘图的最大值
bbox = dict(boxstyle="round", fc="0.8") # 文本框样式

### target_domain=0 的 mdi-PLS 模型
m_diplsT2 = dipls(A=nr_comp, l=l, target_domain=0)
m_diplsT2.fit(Xs, ys, Xs, target_domains)

b_diplsT2 = m_diplsT2.b_
yhat_diplsT2 = m_diplsT2.predict(Xt2)

error_diplsT2 = fct.rmse(yt2, yhat_diplsT2)

# 绘制
plt.figure(figsize=(9, 3))
plt.subplot(131)
plt.scatter(yt2, yhat_diplsT2, color='m', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('target_domain=0')
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.text(4, -1, 'RMSEP = %.2f' % error_diplsT2, bbox=bbox)

### target_domain=1 的 mdi-PLS 模型
m_diplsT2 = dipls(A=nr_comp, l=l, target_domain=1)
m_diplsT2.fit(Xs, ys, Xs, target_domains)

b_diplsT2 = m_diplsT2.b_
yhat_diplsT2 = m_diplsT2.predict(Xt2)

error_diplsT2 = fct.rmse(yt2, yhat_diplsT2)

plt.subplot(132)
plt.scatter(yt2, yhat_diplsT2, color='m', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('target_domain=1')
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.text(4, -1, 'RMSEP = %.2f' % error_diplsT2, bbox=bbox)

### target_domain=2 的 mdi-PLS 模型
m_diplsT2 = dipls(A=nr_comp, l=l, target_domain=2)
m_diplsT2.fit(Xs, ys, Xs, target_domains)

b_diplsT2 = m_diplsT2.b_
yhat_diplsT2 = m_diplsT2.predict(Xt2)

error_diplsT2 = fct.rmse(yt2, yhat_diplsT2)

plt.subplot(133)
plt.scatter(yt2, yhat_diplsT2, color='m', edgecolor='k',alpha=0.75)
plt.plot([min_,max_], [min_,max_], color='k', linestyle=":")
plt.xlim([min_,max_])
plt.ylim([min_,max_])
plt.title('target_domain=2')
plt.xlabel('测量值')
plt.ylabel('预测值')
plt.text(4, -1, 'RMSEP = %.2f' % error_diplsT2, bbox=bbox)

plt.tight_layout()
plt.show()

我们可以看到，当测试数据来自目标域 2 时，使用 target_domain=2 可以获得最佳预测结果（右图）。如果测试数据来自哪个域是未知的，建议使用 target_domain=0（左图）。